# Data Wrangling
In this notebook, datasets from two sources, US Census and US IRS, are imported, cleaned, un-corrupted/empties-filled, and combined with other files.

### Import relevant packages

In [1]:
import numpy as np
import pandas as pd
from glob import glob

## County Business Patterns
The first data source is [County Business Patterns](https://www.census.gov/programs-surveys/cbp.html), an annual census report of number of business establishments, employees, and other metrics by county and county-equivalent in the US. To tidy this data, the desired format would include a multi-index of geo_id (unique county identifier) and year with columns providing values for number of establishments and employees for each NAICS (industry) sector.

### Creating a list of dataframes from each year's csv file

In [2]:
filenames = glob("data/raw/BP_*_00A1.csv")

In [3]:
dataframes = [pd.read_csv(f, encoding='latin-1') for f in filenames]

### Removing punctuation from column names

In [4]:
# remove punctuation from column names
for dataframe in dataframes:
    newcols = []
    for column in dataframe.columns:
        column = column.replace(".","").replace("-","").replace("_","")
        newcols.append(column)
    dataframe.columns = newcols    

### Addressing missing data and schema changes over the years

In [5]:
# Preparing dataframes for each year of county business information
cbp2005_2015 =[]

for dataframe in dataframes:
    
    #rename columns
    dataframe = dataframe[["GEOid2",
                           "GEOdisplaylabel",
                           "GEOfipsstatecode",
                           "GEOfipscountycode",
                           "NAICSdisplaylabel",
                           "ESTAB",
                           "EMP",
                           "YEARid"]]
    dataframe.columns = ["geo_id",
                       "county_name",
                       "state_fips",
                       "county_fips",
                       "naics_industry",
                       "establishments",
                       "employees",
                       "year"]
    
    #exclude state totals
    dataframe = dataframe[dataframe.county_fips != 999]
    
    #exclude puerto rico
    dataframe = dataframe[dataframe.state_fips != 72] 
    
    #exclude retired Alaskan Provinces/Counties
    dataframe = dataframe[dataframe.geo_id != 2201]
    dataframe = dataframe[dataframe.geo_id != 2232]
    dataframe = dataframe[dataframe.geo_id != 2280]
    
    #exclude totals
    dataframe = dataframe[dataframe.naics_industry != "Total for all sectors"]
    
    #apply updated NAICS sector code name
    dataframe.naics_industry = dataframe.naics_industry.str.replace("Mining","Mining, quarrying, and oil and gas extraction")

    #source typo corrections
    dataframe.county_name = dataframe.county_name.str.replace("Dona Ana County, New Mexico","Doña Ana County, New Mexico")
    dataframe.county_name = dataframe.county_name.str.replace("DoÃ±a Ana County, New Mexico","Doña Ana County, New Mexico")
    dataframe.county_name = dataframe.county_name.str.replace("La Salle County, Illinois","LaSalle County, Illinois")
    dataframe.county_name = dataframe.county_name.str.replace("La Salle Parish, Louisiana","LaSalle Parish, Louisiana")    
    
    cbp2005_2015.append(dataframe)
    
    

### Merge now-consistent dataframes into one dataframe with data from each year

In [6]:
cbp = pd.concat(cbp2005_2015)

### Cleanup on the resulting combined dataframe
Missing values for a number of counties were discovered by comparing unique counties from each year.
 - Loving County, TX and Kalawao County, Hawaii are assumed to have no business activity some years
 - New Alasakan counties are created from existing counties, and are best estimated by backfilling the missing data.

In [7]:
#fill in 0s for missing Loving County entries

loving = cbp[cbp.county_name == 'Loving County, Texas']
loving15 = loving.groupby(["geo_id", 
                          "county_name", 
                          "state_fips", 
                          "county_fips", 
                          "naics_industry"])["year", "establishments","employees"]
loving2 = loving15.apply(lambda x: x.set_index("year").reindex(range(2005, 2016), fill_value=0)).reset_index()
loving3 = cbp.merge(loving2, how="outer", on=["year",
                                                  "geo_id",
                                                  "county_name",
                                                  "state_fips",
                                                  "county_fips",
                                                  "naics_industry",
                                                  "establishments",
                                                  "employees"])


#fill in 0s for missing Kalawao County entries

kalawao = loving3[loving3.county_name == 'Kalawao County, Hawaii']
kalawao15 = kalawao.groupby(["geo_id", 
                          "county_name", 
                          "state_fips", 
                          "county_fips", 
                          "naics_industry"])["year", "establishments","employees"]
kalawao2 = kalawao15.apply(lambda x: x.set_index("year").reindex(range(2005, 2016), fill_value=0)).reset_index()
kalawao3 = loving3.merge(kalawao2, how="outer", on=["year",
                                                  "geo_id",
                                                  "county_name",
                                                  "state_fips",
                                                  "county_fips",
                                                  "naics_industry",
                                                  "establishments",
                                                  "employees"])


#backfill the five new alaskan counties

fill_counties = ['Petersburg Census Area, Alaska', 'Skagway Municipality, Alaska', 'Prince of Wales-Hyder Census Area, Alaska', 'Wrangell City and Borough, Alaska', 'Hoonah-Angoon Census Area, Alaska']
dataframeA = kalawao3
    
for fill_county in fill_counties:
    dataframeB = dataframeA[dataframeA.county_name == fill_county].groupby(["geo_id", 
                          "county_name", 
                          "state_fips", 
                          "county_fips", 
                          "naics_industry"])["year", "establishments","employees"].apply(lambda x: x.set_index("year").reindex(range(2005, 2016), method="bfill")).reset_index().merge(dataframeA, how="outer", on=["year",
                                                  "geo_id",
                                                  "county_name",
                                                  "state_fips",
                                                  "county_fips",
                                                  "naics_industry",
                                                  "establishments",
                                                  "employees"])
    dataframeA = dataframeB
    
    
#fill privacy-concerned NaNs with 0s, zip est & emp into a tuple column, drop est & emp cols
dataframeA = dataframeA.fillna(0)


#normalize geo_id, state_fips, and county_fips to consistent lengths
dataframeA["geo_id"] = dataframeA["geo_id"].apply(lambda x: "%05d" % (x,))
dataframeA["state_fips"] = dataframeA["state_fips"].apply(lambda x: "%02d" % (x,))
dataframeA["county_fips"] = dataframeA["county_fips"].apply(lambda x: "%03d" % (x,))


#reshaping dataframe
dataframeA = dataframeA.pivot_table(index=["geo_id", "county_name", "state_fips", "county_fips", "year"], 
                       columns="naics_industry", 
                       values=["establishments", "employees"], 
                       fill_value=0)
businesses_df = dataframeA.reset_index(["state_fips", "county_fips", "county_name"])

## Internal Revenue Service Migration Data
The second data source is [U.S. Population Migration Data](https://www.irs.gov/statistics/soi-tax-stats-migration-data), an annual IRS report that identifies tax filers' filing location for the current and previous year. Separate files are provided for inflow and outflow of tax filers. Furthermore, data is stored in three different formats in the years of interest, including CSV and XLS files.

### Wrangling functions for migration inflow files

In [8]:
def wrangleXLi2005_2009 (xls_year, file):
    #Wrangles one XLS file for county inflows between 2005 and 2009
    
    df = pd.read_excel(file, skiprows=6)


    #construct year 2 geo_id from state and county columns
    df["County"] = df["County"].fillna(0).astype("int64").apply(lambda x: "%03d" % (x,))
    df["y2_geo_id"] =  df["State"] + df["County"]

    #construct year 1 geo_id from state and county columns
    df["State.1"] = df["State.1"].fillna(0).astype("int64").apply(lambda x: "%02d" % (x,))
    df["County.1"] = df["County.1"].fillna(0).astype("int64").apply(lambda x: "%03d" % (x,))
    df["y1_geo_id"] =  df["State.1"] + df["County.1"]


    #drop summary rows
    df = df[df["State.1"] != "96"]
    df = df[df["State.1"] != "97"]
    df = df[df["State.1"] != "98"]

    #drop redundant geo-identifying rows
    df = df.drop(["State", "County", "State.1", "County.1", "State totals, county totals,", "State.2"], axis=1)

    #add year column
    df["year"] = int(xls_year)

    #drop that pesky lingering header line at the top
    df = df[df["gross"] != "income"]

    #rename columns appropriately
    df.columns = ["returns_inflow", "exemptions_inflow", "agi_inflow", "geo_id", "inflow_geo_id", "year"]

    #return everything to numeric types. why did it change in the first place?
    df["returns_inflow"] = pd.to_numeric(df["returns_inflow"], errors="coerce")
    df["exemptions_inflow"] = pd.to_numeric(df["exemptions_inflow"], errors="coerce")
    df["agi_inflow"] = pd.to_numeric(df["agi_inflow"], errors="coerce")

    #cut out the 0s
    df = df[df["geo_id"] != "00000"]
    
    #pivot table to observations of countie migrations by year with origin arrtibutes
    df = df.pivot_table(index=["geo_id", "year"], 
                           columns="inflow_geo_id", 
                           values=["returns_inflow", "exemptions_inflow", "agi_inflow"], 
                           fill_value=0) 
    df = df.fillna(0)
    
    return df

In [9]:
def wrangleXLi2010_2011 (xls_year, file):
    #Wrangles one XLS file for county inflows between 2010 and 2011
    
    df = pd.read_excel(file, skiprows=4, skip_footer=3)


    #construct year 2 geo_id from state and county columns
    df["State Code"] = df["State Code"].fillna(0).astype("int64").apply(lambda x: "%02d" % (x,))
    df["County Code"] = df["County Code"].fillna(0).astype("int64").apply(lambda x: "%03d" % (x,))
    df["y2_geo_id"] =  df["State Code"] + df["County Code"]

    #construct year 1 geo_id from state and county columns
    df["State Code.1"] = df["State Code.1"].fillna(0).astype("int64").apply(lambda x: "%02d" % (x,))
    df["County Code.1"] = df["County Code.1"].fillna(0).astype("int64").apply(lambda x: "%03d" % (x,))
    df["y1_geo_id"] =  df["State Code.1"] + df["County Code.1"]


    #drop summary rows
    df = df[df["State Code.1"] != "96"]
    df = df[df["State Code.1"] != "97"]
    df = df[df["State Code.1"] != "98"]

    #drop redundant geo-identifying rows
    df = df.drop(["State Code", "County Code", "State Code.1", "County Code.1", "State", "County Name"], axis=1)

    #add year column
    df["year"] = int(xls_year)

    #rename columns appropriately
    df.columns = ["returns_inflow", "exemptions_inflow", "agi_inflow", "geo_id", "inflow_geo_id", "year"]

    #return everything to numeric types. why did it change in the first place?
    df["returns_inflow"] = pd.to_numeric(df["returns_inflow"], errors="coerce")
    df["exemptions_inflow"] = pd.to_numeric(df["exemptions_inflow"], errors="coerce")
    df["agi_inflow"] = pd.to_numeric(df["agi_inflow"], errors="coerce")

    #cut out the 0s
    df = df[df["geo_id"] != "00000"]
    
    #pivot table to observations of countie migrations by year with origin arrtibutes
    df = df.pivot_table(index=["geo_id", "year"], 
                           columns="inflow_geo_id", 
                           values=["returns_inflow", "exemptions_inflow", "agi_inflow"], 
                           fill_value=0)
    df = df.fillna(0)
    
    return df

In [10]:
def wrangleCSVi2012_2015 (csv_year, file):
    #Wrangles one CSV file for county inflows between 2012 and 2015
    
    #dataframes = [pd.read_csv(f, encoding='latin-1') for f in filenames]
    
    df = pd.read_csv(file, encoding='latin-1')
    
    #Normalize state and county fips codes to 2 and 3 digits respectively
    df["y1_statefips"] = df["y1_statefips"].apply(lambda x: "%02d" % (x,))
    df["y1_countyfips"] = df["y1_countyfips"].apply(lambda x: "%03d" % (x,))
    df["y2_statefips"] = df["y2_statefips"].apply(lambda x: "%02d" % (x,))
    df["y2_countyfips"] = df["y2_countyfips"].apply(lambda x: "%03d" % (x,))

    #Create geo_id to represent year 1 and year 2 locations
    df["y1_geo_id"] = df["y1_statefips"] + df["y1_countyfips"]
    df["y2_geo_id"] = df["y2_statefips"] + df["y2_countyfips"]

    #drop summary rows
    df = df[df["y1_statefips"] != "96"]
    df = df[df["y1_statefips"] != "97"]
    df = df[df["y1_statefips"] != "98"]

    #replace null values with 0s for computation
    df = df.replace(to_replace=-1, value=0)

    #drop redundant columns
    df = df.drop(["y2_statefips", "y2_countyfips", "y1_statefips", "y1_countyfips", "y1_state", "y1_countyname"], axis=1)

    #add year column
    df["year"] = int(csv_year)
    

    #pivot table to observations of countie migrations by year with origin arrtibutes

    df = df.rename(index=str, columns={"agi":"agi_inflow", "n1":"returns_inflow", "n2":"exemptions_inflow", "y2_geo_id":"geo_id", "y1_geo_id":"inflow_geo_id"})
    
    #cut out the 0s
    df = df[df["geo_id"] != "00000"]
    
    df = df.pivot_table(index=["geo_id", "year"], 
                       columns="inflow_geo_id", 
                       values=["returns_inflow", "exemptions_inflow", "agi_inflow"], 
                       fill_value=0)
    df = df.fillna(0)

    return df

### Wrangling functions for migration outflow files

In [11]:
def wrangleXLo2005_2009 (xls_year, file):
    #Wrangles one XLS file for county outflows between 2005 and 2009
    
    df = pd.read_excel(file, skiprows=6)


    #construct year 2 geo_id from state and county columns
    df["County"] = df["County"].fillna(0).astype("int64").apply(lambda x: "%03d" % (x,))
    df["y2_geo_id"] =  df["State"] + df["County"]

    #construct year 1 geo_id from state and county columns
    df["State.1"] = df["State.1"].fillna(0).astype("int64").apply(lambda x: "%02d" % (x,))
    df["County.1"] = df["County.1"].fillna(0).astype("int64").apply(lambda x: "%03d" % (x,))
    df["y1_geo_id"] =  df["State.1"] + df["County.1"]


    #drop summary rows
    df = df[df["State.1"] != "96"]
    df = df[df["State.1"] != "97"]
    df = df[df["State.1"] != "98"]

    #drop redundant geo-identifying rows
    df = df.drop(["State", "County", "State.1", "County.1", "State totals, county totals,", "State.2"], axis=1)

    #add year column
    df["year"] = int(xls_year)

    #drop that pesky lingering header line at the top
    df = df[df["gross"] != "income"]

    #rename columns appropriately
    df.columns = ["returns_outflow", "exemptions_outflow", "agi_outflow", "geo_id", "outflow_geo_id", "year"]

    #return everything to numeric types. why did it change in the first place?
    df["returns_outflow"] = pd.to_numeric(df["returns_outflow"], errors="coerce")
    df["exemptions_outflow"] = pd.to_numeric(df["exemptions_outflow"], errors="coerce")
    df["agi_outflow"] = pd.to_numeric(df["agi_outflow"], errors="coerce")
    
    #cut out the 0s
    df = df[df["geo_id"] != "00000"]
    
    #pivot table to observations of countie migrations by year with origin arrtibutes
    df = df.pivot_table(index=["geo_id", "year"], 
                           columns="outflow_geo_id", 
                           values=["returns_outflow", "exemptions_outflow", "agi_outflow"], 
                           fill_value=0)
    df = df.fillna(0)
    
    return df

In [12]:
def wrangleXLo2010_2011 (xls_year, file):
    #Wrangles one XLS file for county outflows between 2010 and 2011
    
    df = pd.read_excel(file, skiprows=4, skip_footer=3)


    #construct year 2 geo_id from state and county columns
    df["State Code"] = df["State Code"].fillna(0).astype("int64").apply(lambda x: "%02d" % (x,))
    df["County Code"] = df["County Code"].fillna(0).astype("int64").apply(lambda x: "%03d" % (x,))
    df["y2_geo_id"] =  df["State Code"] + df["County Code"]

    #construct year 1 geo_id from state and county columns
    df["State Code.1"] = df["State Code.1"].fillna(0).astype("int64").apply(lambda x: "%02d" % (x,))
    df["County Code.1"] = df["County Code.1"].fillna(0).astype("int64").apply(lambda x: "%03d" % (x,))
    df["y1_geo_id"] =  df["State Code.1"] + df["County Code.1"]


    #drop summary rows
    df = df[df["State Code.1"] != "96"]
    df = df[df["State Code.1"] != "97"]
    df = df[df["State Code.1"] != "98"]

    #drop redundant geo-identifying rows
    df = df.drop(["State Code", "County Code", "State Code.1", "County Code.1", "State", "County Name"], axis=1)

    #add year column
    df["year"] = int(xls_year)

    #rename columns appropriately
    df.columns = ["returns_outflow", "exemptions_outflow", "agi_outflow", "geo_id", "outflow_geo_id", "year"]

    #return everything to numeric types. why did it change in the first place?
    df["returns_outflow"] = pd.to_numeric(df["returns_outflow"], errors="coerce")
    df["exemptions_outflow"] = pd.to_numeric(df["exemptions_outflow"], errors="coerce")
    df["agi_outflow"] = pd.to_numeric(df["agi_outflow"], errors="coerce")
    
    #cut out the 0s
    df = df[df["geo_id"] != "00000"]

    #pivot table to observations of countie migrations by year with origin arrtibutes
    df = df.pivot_table(index=["geo_id", "year"], 
                           columns="outflow_geo_id", 
                           values=["returns_outflow", "exemptions_outflow", "agi_outflow"], 
                           fill_value=0)
    df = df.fillna(0)

    return df

In [13]:
def wrangleCSVo2012_2015 (csv_year, file):
    #Wrangles one CSV file for county outflows between 2012 and 2015
        
    df = pd.read_csv(file, encoding='latin-1')
    
    #Normalize state and county fips codes to 2 and 3 digits respectively
    df["y1_statefips"] = df["y1_statefips"].apply(lambda x: "%02d" % (x,))
    df["y1_countyfips"] = df["y1_countyfips"].apply(lambda x: "%03d" % (x,))
    df["y2_statefips"] = df["y2_statefips"].apply(lambda x: "%02d" % (x,))
    df["y2_countyfips"] = df["y2_countyfips"].apply(lambda x: "%03d" % (x,))

    #Create geo_id to represent year 1 and year 2 locations
    df["y1_geo_id"] = df["y1_statefips"] + df["y1_countyfips"]
    df["y2_geo_id"] = df["y2_statefips"] + df["y2_countyfips"]

    #drop summary rows
    df = df[df["y2_statefips"] != "96"]
    df = df[df["y2_statefips"] != "97"]
    df = df[df["y2_statefips"] != "98"]

    #replace null values with 0s for computation
    df = df.replace(to_replace=-1, value=0)

    #drop redundant columns
    df = df.drop(["y2_statefips", "y2_countyfips", "y1_statefips", "y1_countyfips", "y2_state", "y2_countyname"], axis=1)

    #add year column
    df["year"] = int(csv_year)
    
    #pivot table to observations of countie migrations by year with origin arrtibutes

    df = df.rename(index=str, columns={"agi":"agi_outflow", "n1":"returns_outflow", "n2":"exemptions_outflow", "y1_geo_id":"geo_id", "y2_geo_id":"outflow_geo_id"})
    
    #cut out the 0s
    df = df[df["geo_id"] != "00000"]
    
    df = df.pivot_table(index=["geo_id", "year"], 
                       columns="outflow_geo_id", 
                       values=["returns_outflow", "exemptions_outflow", "agi_outflow"], 
                       fill_value=0)
    
    df = df.fillna(0)

    return df

### Preparing to ingest files, segmented by file-naming convention

In [14]:
#list of year names using latter 2 digits as 20XX
yearlist1 = ["0405", "0506", "0607"]
yearlist2 = ["0708", "0809"]
yearlist3 = ["0910", "1011"]
yearlist4 = ["1112", "1213", "1314", "1415"]

### Creating a dictionary of years and inflow files
Although the IRS website declares that their format changed in 2012, there are three different file formats and four different file-naming conventions. Part of the complexity comes from the change from trailing to preceding inflow/outflow abbreviation letters competing with recognition of state abbreviations that begin or end with "o" or "i". The implemented solution works around the glob pattern recognition limitations to avoid importing another package.

In [15]:
#glob all inflow filenames for 2005-2009
inflows2005_2009 = {}

for year in yearlist1:
    globstring = "data/raw/countyflow" + year + "/co" + year + "??i.xls"
    glob_year = glob(globstring)
    
    year_formatted = "20" + year[2:4]
    
    inflows2005_2009[year_formatted] = glob_year

for year in yearlist2:
    globstring = "data/raw/countyflow" + year + "/co" + year + "i??.xls"
    glob_year = glob(globstring)
    
    year_formatted = "20" + year[2:4]
    
    inflows2005_2009[year_formatted] = glob_year    
    
    
#glob inflow filenames for 2010 and 2011
inflows2010_2011 = {}

for year in yearlist3:
    globstring = "data/raw/countyflow" + year + "/co" + year + "i??.xls"
    glob_year = glob(globstring)
    
    year_formatted = "20" + year[2:4]
    
    inflows2010_2011[year_formatted] = glob_year    


#glob inflow filenames for 2012-2015
inflows2012_2015 = {}

for year in yearlist4:
    year_formatted = "20" + year[2:4]
    inflows2012_2015[year_formatted] = "data/raw/countyinflow" + year + ".csv"

### Creating a dictionary of years and outflow files

The same process is repeated for outflow files

In [16]:
#glob all outflow filenames for 2005-2009
outflows2005_2009 = {}

for year in yearlist1:
    globstring = "data/raw/countyflow" + year + "/co" + year + "??o.xls"
    glob_year = glob(globstring)
    
    year_formatted = "20" + year[2:4]
    
    outflows2005_2009[year_formatted] = glob_year
    
for year in yearlist2:
    globstring = "data/raw/countyflow" + year + "/co" + year + "o??.xls"
    glob_year = glob(globstring)
    
    year_formatted = "20" + year[2:4]
    
    outflows2005_2009[year_formatted] = glob_year    
    
    
#glob outflow filenames for 2010 and 2011
outflows2010_2011 = {}

for year in yearlist3:
    globstring = "data/raw/countyflow" + year + "/co" + year + "o??.xls"
    glob_year = glob(globstring)
    
    year_formatted = "20" + year[2:4]
    
    outflows2010_2011[year_formatted] = glob_year    


#glob inflow filenames for 2012-2015
outflows2012_2015 = {}

for year in yearlist4:
    year_formatted = "20" + year[2:4]
    outflows2012_2015[year_formatted] = "data/raw/countyoutflow" + year + ".csv"

### Listing wrangled dataframes together

These cells aggregate dataframes for inflows and outflows into lists for concatenation

In [17]:
inflows = []

# yeardict = {"2010": filenames2005_2009["2010"]}

for year, files in inflows2005_2009.items():
    for file in files:
        inflows.append(wrangleXLi2005_2009(year, file))
        
for year, files in inflows2010_2011.items():
    for file in files:
        inflows.append(wrangleXLi2010_2011(year, file))
        
for year, file in inflows2012_2015.items():
    inflows.append(wrangleCSVi2012_2015(year, file))

In [18]:
outflows = []

for year, files in outflows2005_2009.items():
    for file in files:
        outflows.append(wrangleXLo2005_2009(year, file))
        
for year, files in outflows2010_2011.items():
    for file in files:
        outflows.append(wrangleXLo2010_2011(year, file))
        
for year, file in outflows2012_2015.items():
    outflows.append(wrangleCSVo2012_2015(year, file))

### Master dataframes for inflow and outflow

In [19]:
inflow_df = pd.concat(inflows).fillna(0).sort_index()

In [20]:
outflow_df = pd.concat(outflows).fillna(0).sort_index()

### Final dataframes

Over 11 years, 3,142 counties would produce 34,562 records. 

In [21]:
print(inflow_df.shape)
print(outflow_df.shape)
print(businesses_df.shape)

(34564, 9477)
(34563, 9477)
(34562, 45)


In [22]:
inflow_df.head()

agi_inflow                                                     \
inflow_geo_id      01001   01003 01005 01007 01009 01011  01013 01015 01017   
geo_id year                                                                   
01001  2005     714261.0   720.0   0.0   0.0   0.0   0.0    0.0   0.0   0.0   
       2006     756692.0   501.0   0.0   0.0   0.0   0.0  858.0   0.0   0.0   
       2007     827611.0   676.0   0.0   0.0   0.0   0.0    0.0   0.0   0.0   
       2008     901200.0  1086.0   0.0   0.0   0.0   0.0    0.0   0.0   0.0   
       2009     936888.0     0.0   0.0   0.0   0.0   0.0  415.0   0.0   0.0   

                     ...  returns_inflow                                       \
inflow_geo_id 01019  ...           57003 57005 57007 57009  58000 59000 59001   
geo_id year          ...                                                        
01001  2005     0.0  ...             0.0   0.0   0.0  46.0  132.0   572    40   
       2006     0.0  ...             0.0   0.0   0.0  50.0  129.0   613    35   
       2007     0.0  ...             0.0   0.0   0.0  68.0  112.0   599    33   
       2008     0.0  ...             0.0   0.0   0.0  53.0  124.0   619    45   
       2009     0.0  ...             0.0   0.0   0.0  56.0  156.0   601    26   

                                 
inflow_geo_id 59003 59005 59007  
geo_id year                      
01001  2005      85   347   100  
       2006      82   408    88  
       2007      97   354   115  
       2008      91   397    86  
       2009      86   390    99  

[5 rows x 9477 columns]

In [23]:
outflow_df.head()

agi_outflow                                                    \
outflow_geo_id       01001   01003 01005 01007 01009 01011 01013 01015 01017   
geo_id year                                                                    
01001  2005       714261.0   786.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
       2006       756692.0   598.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
       2007       827611.0   761.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
       2008       901200.0     0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
       2009       936888.0  1267.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

                      ...  returns_outflow                                 \
outflow_geo_id 01019  ...            57003 57005 57007 57009  58000 59000   
geo_id year           ...                                                   
01001  2005      0.0  ...              0.0   0.0   0.0  42.0  105.0   501   
       2006      0.0  ...              0.0   0.0   0.0  47.0  107.0   536   
       2007      0.0  ...              0.0   0.0   0.0  42.0  115.0   528   
       2008      0.0  ...              0.0   0.0   0.0  36.0  159.0   553   
       2009      0.0  ...              0.0   0.0   0.0  31.0  123.0   580   

                                        
outflow_geo_id 59001 59003 59005 59007  
geo_id year                             
01001  2005       20    51   344    86  
       2006       27    53   384    72  
       2007       25    71   336    96  
       2008       35    73   356    89  
       2009       25    71   413    71  

[5 rows x 9477 columns]

In [24]:
businesses_df.head()

county_name state_fips county_fips  \
naics_industry                                                   
geo_id year                                                      
01001  2005     Autauga County, Alabama         01         001   
       2006     Autauga County, Alabama         01         001   
       2007     Autauga County, Alabama         01         001   
       2008     Autauga County, Alabama         01         001   
       2009     Autauga County, Alabama         01         001   

                                     employees  \
naics_industry Accommodation and food services   
geo_id year                                      
01001  2005                               1706   
       2006                               1919   
       2007                               2195   
       2008                               2399   
       2009                               2309   

                                                                                         \
naics_industry Administrative and support and waste management and remediation services   
geo_id year                                                                               
01001  2005                                                   280                         
       2006                                                   213                         
       2007                                                   379                         
       2008                                                   200                         
       2009                                                   189                         

                                                           \
naics_industry Agriculture, forestry, fishing and hunting   
geo_id year                                                 
01001  2005                                             0   
       2006                                            36   
       2007                                             0   
       2008                                            45   
       2009                                            31   

                                                                 \
naics_industry Arts, entertainment, and recreation Construction   
geo_id year                                                       
01001  2005                                      0          642   
       2006                                      0          617   
       2007                                    388          563   
       2008                                    236          610   
       2009                                    217          525   

                                                                ...        \
naics_industry Educational services Finance and insurance       ...         
geo_id year                                                     ...         
01001  2005                      57                   273       ...         
       2006                      62                   260       ...         
       2007                      66                   332       ...         
       2008                      58                   329       ...         
       2009                       0                   333       ...         

               establishments                                                 \
naics_industry  Manufacturing Mining, quarrying, and oil and gas extraction    
geo_id year                                                                    
01001  2005                33                                              4   
       2006                30                                              4   
       2007                33                                              4   
       2008                30                                              0   
       2009                31                                              0   

                                                                                     

In [36]:
inflow_df.to_csv("data/interim/inflow.csv")

In [37]:
outflow_df.to_csv("data/interim/outflow.csv")

In [38]:
businesses_df.to_csv("data/interim/businesses.csv")